# Przetwarzanie języka naturalnego w Pythonie

Ten notatnik ma na celu przedstawienie podstawowych kroków, które należy wykonać podczas analizy i klasyfikacji tekstów. W trakcie zadania będziemy tworzyć szereg funkcji pomocniczych, które na koniec wykorzystamy do klasyfikacji krótkich dokumentów tekstowych.

Po wykonaniu tego zadania powinieneś:
+ wiedzieć na czym polega i jak wykonać tokenizację,
+ potrafić wykonać podstawowy stemming,
+ potrafić analizować najpopularniejsze i narzadsze tokeny w zbiorze dokumentów,
+ zdefiniować i usunąć stopwords,
+ wiedzieć jak zamienić zbiór dokumentów na reprezentację bag-of-words,
+ uruchomić wybrany klasyfikator na przetworzonym zbiorze danych.

## Przygotowanie

Na początek trochę bibliotek i przydatnych wyrażeń regularnych:

In [ ]:
import re
import pandas as pd
import nltk

RE_SPACES = re.compile("\s+")
RE_HASHTAG = re.compile("[@#][_a-z0-9]+")
RE_EMOTICONS = re.compile("(:-?\))|(:p)|(:d+)|(:-?\()|(:/)|(;-?\))|(<3)|(=\))|(\)-?:)|(:'\()|(8\))")
RE_HTTP = re.compile("http(s)?://[/\.a-z0-9]+")

Biblioteka [re](https://docs.python.org/2/library/re.html) pozwala definiować wyrażenia regularne, [pandas](http://pandas.pydata.org/) już znasz z ostatnich zajęć, a [nltk](http://www.nltk.org/) to podstawowa biblioteka do przetwarzania języka naturalnego w pythonie. `nltk` to spory zestaw modułów, który domyślnie nie jest instalowany w całości. Aby doinstalować wybrane moduły możesz w interaktywnej konsoli pythona wpisać:

`import nltk
nltk.download()`

Następnie w odpowiednich zakładkach wybrać interesujące moduły. W ramach tego ćwiczenia będziemy wykorzystywać moduł **`punkt`** z zakładki Modules, ale proszę zwrócić uwagę na interesujące korpusy i leksykony, takie jak np. `opinion_lexicon` czy `sentiwordnet`.

Przyda nam się też zbiór danych do testowania kodu.

**Zad. 1: Wczytaj zbiór danych `tweets_train.tsv` do zmiennej `tweets`. Pomiń nagłówek i pozostaw tylko ostatnią kolumnę. Wyświetl pierwsze 6 wierszy, żeby upewnić się, że każdy przykład składa się tylko z tekstu.**

In [ ]:
tweets = pd.read_csv('tweets_train.tsv', error_bad_lines=False, header=None, sep='\t', usecols=[2])
tweets = tweets.rename(columns={2: "text"})

In [ ]:
tweets.head(6)

,text
0,dear @Microsoft the newOoffice for Mac is grea...
1,@Microsoft how about you make a system that do...
2,Not Available
3,Not Available
4,If I make a game as a #windows10 Universal App...
5,"Microsoft, I may not prefer your gaming branch..."


## Tokenizacja i stemming

Żeby uporządkować trochę kod i kolejne kroki, wprowadźmy klasę Tokenizer i BeforeTokenizationNormalizer. Pierwsza będzie służyć jako klasa bazowa dla tworzonych przez tokenizatorów a druga zamieni encje html na poprawne znaki tekstowe.

In [ ]:
class Tokenizer():
    @staticmethod
    def tokenize(text):
        pass
    
class BeforeTokenizationNormalizer():
    @staticmethod
    def normalize(text):
        text = text.strip().lower()
        text = text.replace('&nbsp;', ' ')
        text = text.replace('&lt;', '<')
        text = text.replace('&gt;', '>')
        text = text.replace('&amp;', '&')
        text = text.replace('&pound;', u'£')
        text = text.replace('&euro;', u'€')
        text = text.replace('&copy;', u'©')
        text = text.replace('&reg;', u'®')
        return text

Aby sprawdzić czy wszystko działa - odkomentuj poniższy kod:

In [ ]:
for i in tweets.index:
    tweet = BeforeTokenizationNormalizer.normalize(tweets.text[i])
    print(tweet)

dear @microsoft the newooffice for mac is great and all, but no lync update? c'mon.
@microsoft how about you make a system that doesn't eat my friggin discs. this is the 2nd time this has happened and i am so sick of it!
not available
not available
if i make a game as a #windows10 universal app. will #xboxone owners be able to download and play it in november? @majornelson @microsoft
microsoft, i may not prefer your gaming branch of business. but, you do make a damn fine operating system. #windows10 @microsoft
@mikewolf1980 @microsoft i will be downgrading and let #windows10 be out for almost the 1st yr b4 trying it again. #windows10fail
@microsoft 2nd computer with same error!!! #windows10fail guess we will shelve this until sp1! http://t.co/qcchlkuy8q
just ordered my 1st ever tablet; @microsoft surface pro 3, i7/8gb 512gb ssd. hopefully it works out for dev to replace my laptop =)
after attempting a reinstall, it still bricks, says, "windows cannot finish installing," or somesuch. @m

W kolejnych krokach będziesz implementował coraz bardziej wyrafinowane tokenizatory. Zacznijmy od czegoś w miarę intuicyjnego.

**Zad. 2: Napisz tokenizator, który dzieli słowa według spacji. Tokenizator powinien przyjmować tekst (pojedynczego stringa) i zamieniać go na listę słów. Przetestuj kod za pomocą zakomentowanej pętli.**

In [ ]:
class SimpleTokenizer(Tokenizer):
    @staticmethod
    def tokenize(text):
        words = text.split()
        return words

In [ ]:
for i in tweets.index:
    tweet = BeforeTokenizationNormalizer.normalize(tweets.text[i])
    words_simple = SimpleTokenizer.tokenize(tweet)
    print(words_simple)

['dear', '@microsoft', 'the', 'newooffice', 'for', 'mac', 'is', 'great', 'and', 'all,', 'but', 'no', 'lync', 'update?', "c'mon."]
['@microsoft', 'how', 'about', 'you', 'make', 'a', 'system', 'that', "doesn't", 'eat', 'my', 'friggin', 'discs.', 'this', 'is', 'the', '2nd', 'time', 'this', 'has', 'happened', 'and', 'i', 'am', 'so', 'sick', 'of', 'it!']
['not', 'available']
['not', 'available']
['if', 'i', 'make', 'a', 'game', 'as', 'a', '#windows10', 'universal', 'app.', 'will', '#xboxone', 'owners', 'be', 'able', 'to', 'download', 'and', 'play', 'it', 'in', 'november?', '@majornelson', '@microsoft']
['microsoft,', 'i', 'may', 'not', 'prefer', 'your', 'gaming', 'branch', 'of', 'business.', 'but,', 'you', 'do', 'make', 'a', 'damn', 'fine', 'operating', 'system.', '#windows10', '@microsoft']
['@mikewolf1980', '@microsoft', 'i', 'will', 'be', 'downgrading', 'and', 'let', '#windows10', 'be', 'out', 'for', 'almost', 'the', '1st', 'yr', 'b4', 'trying', 'it', 'again.', '#windows10fail']
['@micro

**Zad. 3: Napisz tokenizator korzystający z funkcji word_tokenize() z biblioteki NLTK.**

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
class NltkTokenizer(Tokenizer):
    @staticmethod
    def tokenize(text):
        # Napisz tokenizator korzystający z funkcji word_tokenize() z biblioteki NLTK.
        # Czy w przypadku tweetów wszystkie słowa zostały poprawnie rozdzielone?
        tokens = nltk.word_tokenize(text)
        return tokens

In [ ]:
for i in tweets.index:
    tweet = BeforeTokenizationNormalizer.normalize(tweets.text[i])
    words_nltk = NltkTokenizer.tokenize(tweet)
    print(words_nltk)

['dear', '@', 'microsoft', 'the', 'newooffice', 'for', 'mac', 'is', 'great', 'and', 'all', ',', 'but', 'no', 'lync', 'update', '?', "c'mon", '.']
['@', 'microsoft', 'how', 'about', 'you', 'make', 'a', 'system', 'that', 'does', "n't", 'eat', 'my', 'friggin', 'discs', '.', 'this', 'is', 'the', '2nd', 'time', 'this', 'has', 'happened', 'and', 'i', 'am', 'so', 'sick', 'of', 'it', '!']
['not', 'available']
['not', 'available']
['if', 'i', 'make', 'a', 'game', 'as', 'a', '#', 'windows10', 'universal', 'app', '.', 'will', '#', 'xboxone', 'owners', 'be', 'able', 'to', 'download', 'and', 'play', 'it', 'in', 'november', '?', '@', 'majornelson', '@', 'microsoft']
['microsoft', ',', 'i', 'may', 'not', 'prefer', 'your', 'gaming', 'branch', 'of', 'business', '.', 'but', ',', 'you', 'do', 'make', 'a', 'damn', 'fine', 'operating', 'system', '.', '#', 'windows10', '@', 'microsoft']
['@', 'mikewolf1980', '@', 'microsoft', 'i', 'will', 'be', 'downgrading', 'and', 'let', '#', 'windows10', 'be', 'out', 'fo

**Zad. 4: Napisz tokenizator, który oprócz standardowych słów obsłuży emitikony i hashtagi. Następnie wykonaj stemming (porter = nltk.PorterStemmer() porter.stem()).**

In [ ]:
class TweetTokenizer(Tokenizer):
    @staticmethod
    def tokenize(text):
        tokens = SimpleTokenizer.tokenize(text)
        i = 0
        while i < len(tokens):
            token = tokens[i]
            match1 = None
            match2 = None
            match3 = None
            match1 = re.search("#[0-9A-Za-z]+", token) #hashtag
            match2 = re.search("^https*://[a-zA-Z0-9./]*", token) #link
            match3 = re.search("(:|;)[)(/A-Za-z]{1}", token) #emotikonka
            # sprawdź czy w ramach tokena występuje emotikona, hashtag lub link
            if match1 is not None or match2 is not None or match3 is not None:
                # wydziel emotikonę lub hashtag jako token a resztę tekstu rozpatrz ponownie
                pass
            else:
                del tokens[i]
                tokens[i:i] = NltkTokenizer.tokenize(token)
            i += 1
 
        # stwórz stemmer i w pętli stemmuj wszystkie tokeny
        porter = nltk.PorterStemmer()
        for i in range(len(tokens)):
          tokens[i] = porter.stem(tokens[i])

        return tokens

In [ ]:
for i in tweets.index:
    tweet = BeforeTokenizationNormalizer.normalize(tweets.text[i])
    words_tweet = TweetTokenizer.tokenize(tweet)
    print(words_tweet)

['dear', '@', 'microsoft', 'the', 'newooffic', 'for', 'mac', 'is', 'great', 'and', 'all', ',', 'but', 'no', 'lync', 'updat', '?', "c'mon", '.']
['@', 'microsoft', 'how', 'about', 'you', 'make', 'a', 'system', 'that', 'doe', "n't", 'eat', 'my', 'friggin', 'disc', '.', 'thi', 'is', 'the', '2nd', 'time', 'thi', 'ha', 'happen', 'and', 'i', 'am', 'so', 'sick', 'of', 'it', '!']
['not', 'avail']
['not', 'avail']
['if', 'i', 'make', 'a', 'game', 'as', 'a', '#windows10', 'univers', 'app', '.', 'will', '#xboxon', 'owner', 'be', 'abl', 'to', 'download', 'and', 'play', 'it', 'in', 'novemb', '?', '@', 'majornelson', '@', 'microsoft']
['microsoft', ',', 'i', 'may', 'not', 'prefer', 'your', 'game', 'branch', 'of', 'busi', '.', 'but', ',', 'you', 'do', 'make', 'a', 'damn', 'fine', 'oper', 'system', '.', '#windows10', '@', 'microsoft']
['@', 'mikewolf1980', '@', 'microsoft', 'i', 'will', 'be', 'downgrad', 'and', 'let', '#windows10', 'be', 'out', 'for', 'almost', 'the', '1st', 'yr', 'b4', 'tri', 'it', '

W wielu zastosowaniach dobrze działa lematyzacja, która również jest dostępna w `nltk`. W tym ćwiczeniu pozostaniemy jednak przy stemmingu.

## Stopwords

W tej części przeanalizujemy częstość występowania różnych słów w korpusie. Interesują nas słowa występujące najczęściej i najrzadziej. Ich analiza pozwoli określić jakie słowa powinny zostać pominięte podczas analizy a jakie powinny zostać połączone w grupy znaczeniowe.

W tym celu przyda nam się jakaś struktura danych do zliczania słów oraz funkcja do rysowania histogramów.

In [ ]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

sns.set(style="whitegrid")
sns.set_color_codes("muted")

def show_histogram(word_counts, title=None):
    plot_df = pd.DataFrame.from_dict(word_counts).rename(columns={0:'Token', 1:'Count'})
    
    f, ax = plt.subplots(figsize=(12, 15))
    p = sns.barplot(x="Count", y="Token", data=plot_df, color="b")
    p.set(xlabel="Count", ylabel="", title=title)

Zacznijmy od zliczenia unikatowych słów. Pomoże nam w tym klasa [Counter](https://docs.python.org/2/library/collections.html#collections.Counter). Poniższy kod wypełnia kolekcję słowami po tokenizacji i stemmingu.

In [ ]:
words = Counter()

for i in tweets.index:
    tweet = BeforeTokenizationNormalizer.normalize(tweets.text[i])
    words.update(TweetTokenizer.tokenize(tweet))

**Zad. 5: Wypisz, zwizualizuj i przeanalizuj 50 najczęściej występujących słów. Czy wszystkie wyświetlone tokeny będą przydatne w dalszej analizie? Czy wszystkie tokeny to słowa?**

In [ ]:
words.most_common(50)

[('.', 399),
 ('the', 395),
 ('@', 305),
 ('amazon', 302),
 (',', 276),
 ('to', 267),
 ('prime', 202),
 ('i', 199),
 ('!', 198),
 ('on', 193),
 ('a', 167),
 ('it', 167),
 ('for', 147),
 ('is', 137),
 ('and', 137),
 ('day', 137),
 ('in', 129),
 ('you', 113),
 ('of', 104),
 ("'s", 102),
 ('...', 101),
 ('?', 100),
 ('may', 100),
 ('friday', 99),
 ('ac/dc', 97),
 ('microsoft', 96),
 ('merkel', 95),
 ('with', 93),
 (':', 92),
 ('my', 90),
 ('not', 90),
 ('be', 88),
 ('have', 88),
 ('angela', 88),
 ('at', 71),
 ('black', 70),
 ('tomorrow', 68),
 ('just', 66),
 ("n't", 65),
 ('avail', 60),
 ('that', 58),
 ('thi', 57),
 ('but', 48),
 ('go', 47),
 ('will', 44),
 ('wa', 43),
 ('see', 43),
 ('get', 42),
 ('from', 40),
 ('``', 37)]

Widać, że nie wszystkie tokeny to słowa. Ponieważ budujemy słownik słów, które będą wykorzystywane do klasyfikacji tekstów, usuńmy podstawowe tokeny, które nie niosą żadnej informacji.

**Zad. 6: Usuń znaki interpunkcyjne z kolekcji words i powtórz analizę. Czy w przypadku badania opinii trzeba wyrzucić wszystkie znaki interpunkcyjne?**

In [ ]:
words = Counter()

for i in tweets.index:
    tweet = BeforeTokenizationNormalizer.normalize(tweets.text[i])
    tweet = re.sub(r'[^\w\s]','',tweet) 
    token = TweetTokenizer.tokenize(tweet)
    
    words.update(token)

In [ ]:
words.most_common(50)

[('the', 396),
 ('amazon', 306),
 ('to', 267),
 ('prime', 202),
 ('on', 193),
 ('a', 166),
 ('it', 164),
 ('for', 147),
 ('i', 147),
 ('and', 136),
 ('day', 136),
 ('is', 132),
 ('in', 129),
 ('you', 104),
 ('of', 104),
 ('may', 99),
 ('microsoft', 97),
 ('friday', 97),
 ('acdc', 97),
 ('merkel', 95),
 ('with', 93),
 ('angela', 91),
 ('not', 90),
 ('my', 89),
 ('be', 88),
 ('have', 84),
 ('at', 72),
 ('black', 70),
 ('tomorrow', 68),
 ('just', 65),
 ('avail', 60),
 ('that', 57),
 ('thi', 57),
 ('but', 48),
 ('go', 48),
 ('will', 44),
 ('see', 43),
 ('wa', 42),
 ('get', 41),
 ('from', 41),
 ('if', 37),
 ('your', 36),
 ('im', 34),
 ('so', 32),
 ('out', 32),
 ('me', 31),
 ('by', 31),
 ('about', 30),
 ('deal', 30),
 ('order', 29)]

Wyraźnie widać, że nawet bez znaków intepunkcyjnych pozostało dużo zbędnych tokenów. Poniżej lista słów, które często są oznaczane jako tzw. stopwords, czyli słowa występujące często a nie niosące informacji.

In [ ]:
stopwords = ["a", "about", "after", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been",
            "before", "being", "between", "both", "by", "could", "did", "do", "does", "doing", "during", "each",
            "for", "from", "further", "had", "has", "have", "having", "he", "her", "here", "hers", "herself", "him",
            "himself", "his", "how", "i", "in", "into", "is", "it", "its", "itself", "let", "me", "more", "most", "my",
            "myself", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "own", "sha",
            "she", "should", "so", "some", "such", "than", "that", "the", "their", "theirs", "them", "themselves",
            "then", "there", "there's", "these", "they", "this", "those", "through", "to", "until", "up", "very",
            "was", "we", "were", "what", "when", "where", "which", "while", "who","whom", "with", "would", "you",
            "your", "yours", "yourself", "yourselves",
            "n't", "'s", "'ll", "'re", "'d", "'m", "'ve",
            "above", "again", "against", "below", "but", "cannot", "down", "few", "if", "no", "nor", "not", "off",
            "out", "over", "same", "too", "under", "why"]

**Zad. 7: Korzystając z listy `sotpwords` usuń z kolekcji `words` popularne stopwords i ponów analizę. Czy wszystkie stopwords zawsze należy wyrzucać? Czy słowa takie jak "not" mogą być nośnikiem opinii? Jeśli masz jeszcze dużo czasu, możesz przeanalizować listę słów w nltk.corpus.stopwords.words('english')?**

In [ ]:
words = Counter()

for i in tweets.index:
    tweet = BeforeTokenizationNormalizer.normalize(tweets.text[i])
    tweet = re.sub(r'[^\w\s]','',tweet) 
    token = TweetTokenizer.tokenize(tweet)
    token = [x for x in token if x not in stopwords]
    words.update(token)

In [ ]:
words.most_common(50)

[('amazon', 306),
 ('prime', 202),
 ('day', 136),
 ('may', 99),
 ('microsoft', 97),
 ('friday', 97),
 ('acdc', 97),
 ('merkel', 95),
 ('angela', 91),
 ('black', 70),
 ('tomorrow', 68),
 ('just', 65),
 ('avail', 60),
 ('thi', 57),
 ('go', 48),
 ('will', 44),
 ('see', 43),
 ('wa', 42),
 ('get', 41),
 ('im', 34),
 ('deal', 30),
 ('order', 29),
 ('ha', 28),
 ('sale', 28),
 ('time', 27),
 ('new', 25),
 ('1st', 24),
 ('one', 24),
 ('night', 24),
 ('now', 23),
 ('free', 23),
 ('can', 22),
 ('2nd', 21),
 ('got', 21),
 ('look', 21),
 ('dont', 21),
 ('3rd', 20),
 ('today', 20),
 ('like', 20),
 ('whi', 19),
 ('come', 19),
 ('say', 18),
 ('wednesday', 18),
 ('want', 18),
 ('concert', 18),
 ('make', 17),
 ('cant', 17),
 ('thursday', 17),
 ('germani', 17),
 ('us', 16)]

Oprócz tokenów, które występują zbyt często, problemem bywają również najrzadsze tokeny.

**Zad. 8: Przeanalizuj 100 listę najrzadszych tokenów. Czy wszystkie tokeny są rzeczywiście rzadkie? Czy jakieś typy tokenów powtarzają się? Jak myślisz, co najlepiej zrobić z takimi tokenami jak 1, 2, 3... 10%, 20%, 30%..., 12:00, 19:50, 22:30, ..., 2005, 2010, 1995..., 10\$, 20\$, 30\$... ?**

In [ ]:
words.most_common()[-100:][::-1]

[('columnista', 1),
 ('invas', 1),
 ('organis', 1),
 ('freedomoffreedo', 1),
 ('reaganitegop', 1),
 ('piti', 1),
 ('restor', 1),
 ('refugeeswelcom', 1),
 ('httptcoooqdqdbwit', 1),
 ('bold', 1),
 ('inclin', 1),
 ('shown', 1),
 ('rare', 1),
 ('xenophob', 1),
 ('action', 1),
 ('critic', 1),
 ('concern', 1),
 ('citizen', 1),
 ('httptcocvh6nkcxoq', 1),
 ('httptcoled4qywrr', 1),
 ('httptcod7il2qp6di', 1),
 ('humanitarian', 1),
 ('graduat', 1),
 ('httptco9g3het8dzd', 1),
 ('newsfusionapp', 1),
 ('must', 1),
 ('whole', 1),
 ('httptcogvvyrjpcop', 1),
 ('migrantcrisi', 1),
 ('migrantmarch', 1),
 ('httptcowq1s3fxdd2', 1),
 ('exodu', 1),
 ('ran', 1),
 ('nobodi', 1),
 ('nishelo', 1),
 ('smith_rfkennedi', 1),
 ('httptcogral0rmwv4', 1),
 ('exampl', 1),
 ('httptcolsi57oqqi0', 1),
 ('sweden', 1),
 ('httptcoz4d0iflavq', 1),
 ('httptcoifvnbuqfgf', 1),
 ('colour', 1),
 ('httpstcok5xwf52hfm', 1),
 ('gunboat', 1),
 ('sink', 1),
 ('favour', 1),
 ('kthopkin', 1),
 ('httptco4lqn0jeyrr', 1),
 ('snp', 1),
 ('spa

Na tym etapie w zmiennej `words` mamy wstępnie przygotowany słownik. Jak widać słownik możnaby jeszcze ulepszyć, ale w tym ćwiczeniu poprzestaniemy na tym co do tej pory zrobiliśmy.

## Klasyfikacja

Mając słownik zamienimy przykłady uczące na reprezentację bag of words (BOW). W tej reprezentacji każdy przykład uczący (pojedynczy dokument tekstowy) przyjmuje formę wektora liczb. Każda pozycja w wektorze oznacza kolejne słowo ze słownika, a liczba na danej pozycji mówi czy (0-1) lub jak często (0-k) dane słowo występuje w tekście. Taka reprezentacja wektorowa ma tę zaletę, że można ją łatwo rozszerzać o dodatkowe atrybuty.

**Zad. 9: Przeanalizuj poniższy kod tworzący reprezentację bag-of-words. Co to jest `csr_matrix`? Czy wykorzystanie tej klasy jest konieczne?**

In [ ]:
def create_bow(documents, features):
    row = []
    col = []
    data = []

    labels = []

    for i in documents.index:
        tweet = BeforeTokenizationNormalizer.normalize(documents.iloc[i, 2])
        label = documents.iloc[i, 1]
        tweet_tokens = TweetTokenizer.tokenize(tweet)

        labels.append(label)
        for token in set(tweet_tokens):
            if token not in features:
                continue
            row.append(i)
            col.append(features[token])
            data.append(1)
    return csr_matrix((data, (row, col)), shape=(len(documents), len(features))), labels

Na koniec wykorzystajmy dotychczasową wiedzę, żeby nauczyć i przetestować klasyfikator.

**Zad. 10: Sprawdź działanie klasyfikatora Random Forest na przetworzonych wcześniej danych. Jak na trafność klasyfikacji wpływa parametr min_word_count?**

In [ ]:
from scipy.sparse import csr_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score

min_word_count = 5

train_tweets = pd.read_csv("tweets_train.tsv", sep="\t", header=None)
test_tweets = pd.read_csv("tweets_test.tsv", sep="\t", header=None)
common_words = list([k for k, v in words.most_common() if v > min_word_count])

feature_dict = {}
for word in common_words:
    feature_dict[word] = len(feature_dict)

print("Training classifier...")
X_train, y_train = create_bow(train_tweets, feature_dict)
list_of_labels = list(set(y_train))
classifier = RandomForestClassifier(n_estimators=300, n_jobs=-1, random_state=23)
classifier.fit(X_train, y_train)

print("Testing...")
test_tweets = pd.read_csv("tweets_test.tsv", sep="\t", header=None)
X_test, y_test = create_bow(test_tweets, feature_dict)
predicted = classifier.predict(X_test)

print("=================== Results ===================")
print("            Positive    Neutral     Negative   ")
print("F1       ", f1_score(y_test, predicted, average=None, pos_label=None, labels=list_of_labels))
print("Precision", precision_score(y_test, predicted, average=None, pos_label=None, labels=list_of_labels))
print("Recall   ", recall_score(y_test, predicted, average=None, pos_label=None, labels=list_of_labels))

Training classifier...
Testing...
=================== Results ===================
            Positive    Neutral     Negative   
F1        [0.25       0.72058824 0.33333333]
Precision [0.33333333 0.66666667 0.3877551 ]
Recall    [0.2        0.784      0.29230769]
